In [44]:
from datasets import load_dataset

import pandas as pd
import numpy as np
import re

In [45]:
nsmc = load_dataset("Blpeng/nsmc")

df_nsmc = pd.concat([
    pd.DataFrame(nsmc["train"]),
    pd.DataFrame(nsmc["test"])
])

df_nsmc = df_nsmc.rename(columns={"document": "text", "label": "sentiment"})
df_nsmc = df_nsmc.drop(["Unnamed: 0", "id"], axis = 1)
df_nsmc["source"] = "nsmc"

df_nsmc.head()

Repo card metadata block was not found. Setting CardData to empty.


,text,sentiment,source
0,아 더빙.. 진짜 짜증나네요 목소리,0,nsmc
1,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1,nsmc
2,너무재밓었다그래서보는것을추천한다,0,nsmc
3,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0,nsmc
4,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1,nsmc


In [46]:
print(df_nsmc.shape) 
print(df_nsmc.columns)

(200000, 3)
Index(['text', 'sentiment', 'source'], dtype='object')


In [76]:
df_nsmc_positive = df_nsmc[df_nsmc["sentiment"] == 1]
df_nsmc_negative = df_nsmc[df_nsmc["sentiment"] == 0]

print(df_nsmc_positive.info())
print(df_nsmc_negative.info())

<class 'pandas.core.frame.DataFrame'>
Index: 99996 entries, 1 to 49995
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   text       99996 non-null  object
 1   sentiment  99996 non-null  int64 
 2   source     99996 non-null  object
dtypes: int64(1), object(2)
memory usage: 3.1+ MB
None
<class 'pandas.core.frame.DataFrame'>
Index: 99996 entries, 0 to 49999
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   text       99996 non-null  object
 1   sentiment  99996 non-null  int64 
 2   source     99996 non-null  object
dtypes: int64(1), object(2)
memory usage: 3.1+ MB
None


In [85]:
df_nsmc_positive = df_nsmc_positive.sample(50000).reset_index(drop = True)
df_nsmc_negative = df_nsmc_negative.sample(50000).reset_index(drop = True)

df_nsmc = pd.concat([df_nsmc_positive, df_nsmc_negative], axis = 0)
df_nsmc = df_nsmc.sample(frac = 1).reset_index(drop = True)

df_nsmc.head()

,text,sentiment,source
0,내용이 무슨 내용인지도 모르겠고... 초반에 할머니 대사할때 자막은 왜 넣었는지 모...,0,nsmc
1,"이건 영화도 아니고, 시트콤도 아녀~",0,nsmc
2,완!전!잼!없!음!,0,nsmc
3,헐리웃도 유행타면 개나 소나 만드는건 비슷하구나..,0,nsmc
4,1점도 아깝네ㅡㅡ 답답하고 지루하고 감동도 없고 교훈도 없는 영화,0,nsmc


In [47]:
df_nsmc.dropna(inplace = True)
df_nsmc.shape

(199992, 3)

In [48]:
df_kor = pd.read_excel("한국어_연속적_대화_데이터셋.xlsx")
df_kor.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,행복,중립,슬픔,공포,혐오,분노,놀람
0,dialog #,발화,감정,NaN,NaN,1030.0,NaN,NaN,NaN,NaN,NaN,NaN
1,S,아 진짜! 사무실에서 피지 말라니깐! 간접흡연이 얼마나 안좋은데!,분노,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,그럼 직접흡연하는 난 얼마나 안좋겠니? 안그래? 보면 꼭... 지 생각만 하고.,혐오,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,손님 왔어요.,중립,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,손님? 누구?,중립,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [49]:
df_kor.drop(["Unnamed: 0", "Unnamed: 3", "Unnamed: 4", "행복", "중립", "슬픔", "공포", "혐오", "분노", "놀람"], 
            axis = 1,
            inplace = True
)
df_kor.drop(0, axis = 0, inplace = True)
df_kor.columns

Index(['Unnamed: 1', 'Unnamed: 2'], dtype='object')

In [50]:
df_kor.dropna(axis = 0, inplace = True)
df_kor.isnull().sum()

Unnamed: 1    0
Unnamed: 2    0
dtype: int64

In [51]:
df_kor.rename(columns = {"Unnamed: 1" : "text", "Unnamed: 2" : "sentiment"}, inplace = True)
df_kor["source"] = "aihub_data"
df_kor.head()

,text,sentiment,source
1,아 진짜! 사무실에서 피지 말라니깐! 간접흡연이 얼마나 안좋은데!,분노,aihub_data
2,그럼 직접흡연하는 난 얼마나 안좋겠니? 안그래? 보면 꼭... 지 생각만 하고.,혐오,aihub_data
3,손님 왔어요.,중립,aihub_data
4,손님? 누구?,중립,aihub_data
5,몰라요. 팀장님 친구래요.,중립,aihub_data


In [52]:
neutrality = df_kor[df_kor["sentiment"] == "중립"].index
df_kor.drop(index = neutrality, axis = 0, inplace = True)

print(df_kor.shape)
print(df_kor.head())

(11835, 3)
                                            text sentiment      source
1           아 진짜! 사무실에서 피지 말라니깐! 간접흡연이 얼마나 안좋은데!        분노  aihub_data
2   그럼 직접흡연하는 난 얼마나 안좋겠니? 안그래? 보면 꼭... 지 생각만 하고.        혐오  aihub_data
7                                 그걸 내가 어떻게 알아요!        분노  aihub_data
15                          동사무소에서 인구조사 나왔니? 지금.        분노  aihub_data
17                               나? ... 나보고 하라고?        놀람  aihub_data


In [53]:
df_kor["sentiment"].value_counts().index.tolist()

drop_list = ['ㅍ', '분', 'ㅈ중립', '분ㄴ', '중림', 'ㄴ중립', '줄']
df_kor = df_kor[~df_kor["sentiment"].isin(drop_list)]

value_series = df_kor["sentiment"].value_counts()
value_series

sentiment
놀람    4866
분노    3628
슬픔    1972
행복    1030
혐오     220
공포      98
Name: count, dtype: int64

In [54]:
negative_word_list = ["분노", "슬픔", "혐오", "공포"]

cnt = value_series[value_series.index.isin(negative_word_list)].sum()
cnt

np.int64(5918)

In [55]:
ratio = []
for word in negative_word_list:
    ratio.append(round((value_series[word] / cnt) * 1000))

df_n_list = []
for ratio, word in zip(ratio, negative_word_list):
    df_n_list.append(df_kor[df_kor["sentiment"] == word].sample(n = ratio, replace = False, random_state = 42))

df_n = pd.concat(df_n_list).reset_index(drop = True)
df_n["sentiment"].value_counts()

sentiment
분노    613
슬픔    333
혐오     37
공포     17
Name: count, dtype: int64

In [56]:
df_p = df_kor[df_kor["sentiment"] == "행복"].reset_index(drop = True)

In [57]:
df_aihub = pd.concat([df_p, df_n], axis=0).sample(frac=1, random_state=42).reset_index(drop=True)

In [58]:
df_aihub["sentiment"] = df_aihub["sentiment"].apply(lambda x: 1 if "행복" in x else 0)

In [59]:
df_huggingface = load_dataset("Dasool/KoMultiText")
df_huggingface

DatasetDict({
    train: Dataset({
        features: ['comment', 'preference', 'profanity', 'gender', 'politics', 'nation', 'race', 'region', 'generation', 'social_hierarchy', 'appearance', 'others'],
        num_rows: 38361
    })
    test: Dataset({
        features: ['comment', 'preference', 'profanity', 'gender', 'politics', 'nation', 'race', 'region', 'generation', 'social_hierarchy', 'appearance', 'others'],
        num_rows: 2000
    })
})

In [60]:
df_hug_dc = df_huggingface["train"].to_pandas()

In [61]:
df_hug_dc.columns

Index(['comment', 'preference', 'profanity', 'gender', 'politics', 'nation',
       'race', 'region', 'generation', 'social_hierarchy', 'appearance',
       'others'],
      dtype='object')

In [62]:
drop_index = df_hug_dc[df_hug_dc["preference"] == 2].index

df_hug_dc.drop(index = drop_index, axis = 0, inplace = True)

In [63]:
df_hug_dc["preference"].value_counts().sort_index()

preference
0     4306
1    11886
3     2186
4      683
Name: count, dtype: int64

In [64]:
df_hug_dc.drop(columns = ['profanity', 'gender', 'politics', 'nation',
       'race', 'region', 'generation', 'social_hierarchy', 'appearance',
       'others'], axis = 1, inplace = True)

df_hug_dc.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19061 entries, 0 to 38360
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   comment     19061 non-null  object
 1   preference  19061 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 446.7+ KB


In [65]:
df_hug_dc.drop(df_hug_dc[df_hug_dc["preference"] == 1].index, axis = 0, inplace = True)
df_hug_dc["preference"].value_counts()

preference
0    4306
3    2186
4     683
Name: count, dtype: int64

In [66]:
df_hug_dc["preference"] = df_hug_dc["preference"].replace({3 : 1, 4 : 1, 0 : 0})
df_hug_dc["preference"].value_counts()

preference
0    4306
1    2869
Name: count, dtype: int64

In [67]:
df_hug_dc = df_hug_dc[~df_hug_dc['comment'].apply(lambda x: bool(re.search(r'[A-Za-z]', x)))]
df_hug_dc.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6839 entries, 0 to 38360
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   comment     6839 non-null   object
 1   preference  6839 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 160.3+ KB


In [74]:
df_hug_dc.rename(columns = {"comment" : "text", "preference" : "sentiment"}, inplace = True)
df_hug_dc["source"] = "hugggingface"

In [86]:
# df_nsmc
# df_aihub
# df_hug_dc

print(df_nsmc.info())
print(df_aihub.info())
print(df_hug_dc.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 3 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   text       100000 non-null  object
 1   sentiment  100000 non-null  int64 
 2   source     100000 non-null  object
dtypes: int64(1), object(2)
memory usage: 2.3+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2030 entries, 0 to 2029
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   text       2030 non-null   object
 1   sentiment  2030 non-null   int64 
 2   source     2030 non-null   object
dtypes: int64(1), object(2)
memory usage: 47.7+ KB
None
<class 'pandas.core.frame.DataFrame'>
Index: 6839 entries, 0 to 38360
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   text       6839 non-null   object
 1   sentiment  6839 non-null   int64 
 2   source     6839 

In [98]:
final_df = pd.concat([df_nsmc, df_aihub, df_hug_dc], axis = 0)
final_df = final_df.sample(frac = 1).reset_index(drop = True)

final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 108869 entries, 0 to 108868
Data columns (total 3 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   text       108869 non-null  object
 1   sentiment  108869 non-null  int64 
 2   source     108869 non-null  object
dtypes: int64(1), object(2)
memory usage: 2.5+ MB


In [99]:
final_df.to_csv("final_df.csv")